Osnabrück University - Computer Vision (Winter Term 2022/23) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 03: Morphological Operations¶

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, November 27, 2022**. If you need help (and Google and other resources were not enough) use the StudIP forum. Please upload your results to your group's Stud.IP folder.

## Assignment 0: Math recap (complex numbers) [0 Points]

This exercise is supposed to be very easy, does not give any points, and is voluntary. There will be a similar exercise on every sheet. It is intended to revise some basic mathematical notions that are assumed throughout this class and to allow you to check if you are comfortable with them. Usually you should have no problem to answer these questions offhand, but if you feel unsure, this is a good time to look them up again. You are always welcome to discuss questions with the tutors or in the practice session. Also, if you have a (math) topic you would like to recap, please let us know.

**a)** What is a *complex number*, what is the *complex plane*, how are complex numbers usually denoted?

The complex numbers, usually denoted by $\mathbb{C}$, are the algebraic closure of the real numbers $\mathbb{R}$, meaning that every non-constant polynomial has some root in $\mathbb{C}$. This is realized by adding "artificial" numbers to provide the necessary roots missing in $\mathbb{R}$, e.g. a new number $i$ with $i^2=-1$, as a root for the polynomial $X^2+1$. In fact, all complex numbers can be written as $a+b\cdot i$ with some $a,b\in\mathbb{R}$.

The complex plane is a geometric embedding of complex numbers into the two-dimensional Euclidean plane, mapping the number $a+b\cdot i$ on the point $(a,b)$. It was a major breakthrough towards the acceptance of complex numbers, as this embedding allows to give geometric interpretations to arithmetic operations like sum and product.

**b)** What is the *real* and the *imaginary* part of a complex number? What is the *absolute value* of a complex number? What is the *complex conjugate*?

Given a complex number $c=a+b\cdot i$, the real number $a = \Re c$ is called the real part and $b=\Im c$ is called the imaginary part of $c$. In fact, the real numbers can be seen as the subset of real numbers whose imaginary part is $0$.

The absolute value of $c$ is just the Euclidean norm of $c$ in the complex plane, i.e., its distance from the origin: $|c| = \sqrt{(\Re c)^2 + (\Im c)^2}$

The complex conjugate of a complex number is obtained by changing the sign of its imaginary part: $\bar{c} = \Re c - \Im c$. Hence the conjugate of a real number is just that number.

**c)** What are polar coordinates? What are their advantages? Can you convert between cartesian and polar coordinates? Can you write down $i=\sqrt{-1}$ in polar coordinates? What about $\sqrt{i}$?

Polar coordinates represent the position in a plane as a pair $(r, \phi)$, where $r$ denotes the distance from the center and $\phi$ the angle. Polar coordinates for complex numbers have the advantage, that multiplication (and exponents) get a very simple gemetrical meaning. The product of two numbers (in polar coordinates) is formed as

$$(r_1,\phi_1)\cdot(r_2,\phi_2) = (r_1r_2,\phi_1+\phi_2)$$

that is, distances are multiplied while angles are sumed up. This leads to the following simple rule for exponents (which even holds for fractional $n$):

$$(r,\phi)^n = (r^n,n\cdot \phi)$$

Hence we get the following polar coordinates for $i$ (note that the polar coordinates for $-1$ are $(1,\pi))$:

$$i = \sqrt{-1} = (1,\pi)^{\tfrac12} = (1,\tfrac12\pi)$$

and further

$$\sqrt{i} = (1,\tfrac12\pi)^{\tfrac12} = (1,\tfrac14\pi)$$

From the polar coordinates we can obtain cartesian coordinates as

$$\Re c= r\cdot\cos(\phi)\qquad \text{and}\qquad \Im c=r\cdot\sin(\phi)$$

So we get:
$$\sqrt{i} = \cos(\frac14\pi)+i\cdot\sin(\frac14\pi) = \frac{\sqrt{2}}{2}(1+i)$$

**d)** Python, and also numpy, support calculations with complex numbers. Consult the documentation to find out details. Notice that $i$ is substituted by $j$ in Python.

In [ ]:
a = 3 + 4j
b = complex(3, 4)
print("a = {}, type(a) = {}".format(a, type(a)))
print("b = {}, type(b) = {}".format(b, type(b)))
print("real(a) = {}, imag(a) = {}".format(a.real,a.imag))
print("abs(a) = {}".format(abs(a)))
print("conjugate(a) = {}".format(a.conjugate()))
print("i*i = {}".format(1j**2))

# normal "math" functions are non complex:
print("-- the math module ---------------------------")
import math
#print(math.sqrt(-1)) # ValueError: math domain error
#print(math.sin(3+4j)) # TypeError: can't convert complex to float
#print(math.exp(2*math.pi*1j)) # TypeError: can't convert complex to float

# but there is a "cmath" module
print("-- the cmath module --------------------------")
import cmath
print(cmath.sqrt(-1))
print(cmath.sin(3+4j))
print(cmath.exp(2*math.pi*1j))

# numpy has its own types for complex numbers (np.complex64 and np.complex128)
print("-- numpy -------------------------------------")
import numpy as np
a = np.complex64(3+4j)
print("a = {}, type(a) = {}".format(a, type(a)))
print("a*a = {}".format(a**2))
print("real(a) = {}, imag(a) = {}".format(a.real,a.imag))
print("abs(a) = {}".format(abs(a)))
print("conjugate(a) = {}".format(a.conjugate()))

# numpy supports computation with complex numbers (provided the input arguments are complex)
print(np.sqrt(-1)) # Warning: invalid value encountered in sqrt
print(np.sqrt(-1+0j))
print(np.sin(3+4j))
print(np.exp(2*np.pi*1j))

## Assignment 1: Properties of morphological operators (5 points)

This exercise will elaborate on the basic morphological operators of *erosion* and *dilation* (cf. CV-05 slides 4-14).

### a) Duality

Proof that *erosion* and *dilation* are *dual* operators, i.e.

$$ g^{\ast}\oplus S = (g\ominus S)^{\ast}\qquad\text{and}\qquad
g^{\ast}\ominus S = (g\oplus S)^{\ast}$$

here $g^{\ast}$ denotes the inverted binary image, i.e. $g^{\ast}(x,y) = 1 - g(x,y) = \neg g(x,y)$, i.e. 1-pixel become 0 and 0-pixel become 1.

Intuitively, this can be expressed by saying that extending the fringe of an object is "eating" from the fringe of its complement. 

Formally, one can write
\begin{align}
(g^{\ast}\oplus S)(x,y)
& = \bigvee_{i\in[-m,m]} \bigvee_{i\in[-n,n]} S(i+m, j+n)\wedge g^{\ast}(x+i,y+j) \\
& = \bigvee_{i\in[-m,m]} \bigvee_{i\in[-n,n]} \neg\neg S(i+m, j+n)\wedge \neg g(x+i,y+j) \\
& = \bigvee_{i\in[-m,m]} \bigvee_{i\in[-n,n]} \neg(\neg S(i+m, j+n)\vee g(x+i,y+j)) \\
& = \bigvee_{i\in[-m,m]} \bigvee_{i\in[-n,n]} \neg(S(i+m, j+n)\to g(x+i,y+j)) \\
& = \neg\bigwedge_{i\in[-m,m]} \bigwedge_{i\in[-n,n]} (S(i+m, j+n)\to g(x+i,y+j)) \\
&= \neg(g\ominus S)(x,y) \\
&= (g\ominus S)^{\ast}(x,y)
\end{align}

The other case is analogous.


Alternative Notation: one can shorten the proof a bit by introducing a simplified notation: letting $(i,j)\in S$ mean to iterate over all $(i,j)$ for which $S(i+m, j+n)$ is true (a $1$-element), we can rewrite the definition of dilation from the slides as follows:
$$g'(x,y) = \bigvee_{i\in[-m,m]} \bigvee_{i\in[-n,n]}S(i+m, i+n)\wedge g(x+i, y+j)
= \bigvee_{(i,j)\in S} g(x+i, y+j)$$
and erosion as
$$g'(x,y) = \bigwedge_{i\in[-m,m]} \bigwedge_{i\in[-n,n]}(S(i+m, i+n)\to g(x+i, y+j))
= \bigwedge_{(i,j)\in S} g(x+i, y+j)$$
(using the convention that $\bigvee_\emptyset=0$ and $\bigwedge_\emptyset=1$). Using that notation, we can simply derive
\begin{align}
(g^{\ast}\oplus S)(x,y)
& = \bigvee_{(i,j)\in S} g^{\ast}(x+i,y+j) \\
& = \bigvee_{(i,j)\in S} \neg g(x+i,y+j) \\
& = \neg\bigwedge_{(i,j)\in S} g(x+i,y+j) \\
&= \neg(g\ominus S)(x,y) \\
&= (g\ominus S)^{\ast}(x,y)
\end{align}


### b) Superposition

As *erosion* and *dilation* have been introduced for binary images, the notion of *linearity* is not really appropriate here. However, some weaker version, called *superposition* can be defined: instead of forming a linear combination, one takes the logical disjunction:

$$(g_1\lor g_2)(x,y) := g_1(x,y)\lor g_2(x,y)$$

Check for both operations if *erosion* and *dilation* are "compatible" with superposition, i.e. if first *eroding* (or *dilating*) two images and superposing the result is the same as first superposing the images and then *eroding* (or *dilating*) the result.

The superposition principle holds for *dilation*, as we are just dealing with disjunctions: 

\begin{align*}
    ((g_1\lor g_2)\oplus S)(x,y)
    & = \bigvee_{(i,j)\in S} (g_1\lor g_2)(x+i,y+j) \\
    & = \bigvee_{(i,j)\in S} \left(g_1(x+i,y+j) \lor g_2(x+i,y+j)\right) \\
    & = \left(\bigvee_{(i,j)\in S} g_1(x+i,y+j)\right) \lor \left(\bigvee_{(i,j)\in S}g_2(x+i,y+j)\right) \\
    & = (g_1\oplus S)(x,y)\lor (g_2\oplus S)(x,y)
\end{align*}


For erosion, the superposition principle does not hold in general: Example (with zero padding): 

$$
g_1 = \left(
\begin{array}{lll}
1 & 0 & 1 \end{array}\right)\qquad 
g_2 = \left(
\begin{array}{lll}
0 & 1 & 0 \end{array}\right)\qquad 
S = \left( \begin{array}{lll} 1 & 1 & 1 \end{array} \right)
$$

$$
\begin{align*}
    (g_1\lor g_2)\ominus S)(x,y) 
    & = \left( \left( \begin{array}{lll} 1 & 0 & 1 \end{array} \right) \lor
        \left( \begin{array}{lll} 0 & 1 & 0 \end{array} \right) \right) \ominus
        \left( \begin{array}{lll} 1 & 1 & 1 \end{array} \right) \\
    & = \left( \begin{array}{lll} 1 & 1 & 1 \end{array} \right) \ominus
        \left( \begin{array}{lll} 1 & 1 & 1 \end{array} \right) \\
    & = \left( \begin{array}{lll} 0 & 1 & 0 \end{array} \right) \\
    (g_1\ominus S)(x,y)  \lor (g_2\ominus S)(x,y) 
    & = \left( \left( \begin{array}{lll} 1 & 0 & 1 \end{array} \right) \ominus
        \left( \begin{array}{lll} 1 & 1 & 1 \end{array} \right) \right) \lor
        \left( \left( \begin{array}{lll} 0 & 1 & 0 \end{array} \right) \ominus
        \left( \begin{array}{lll} 1 & 1 & 1 \end{array} \right) \right) \\
    & = \left( \begin{array}{lll} 0 & 0 & 0 \end{array} \right) \lor
        \left( \begin{array}{lll} 0 & 0 & 0 \end{array} \right) \\
    & = \left( \begin{array}{lll} 0 & 0 & 0 \end{array} \right)
\end{align*}
$$


But one can show a weaker form, namely

$$(g_1\lor g_2)\ominus S \to (g_1\ominus S)\lor (g_2\ominus S)$$

holds. That is, the erosion of the combination of two images is in general larger than the combination of two eroded images. The following cell shows an example of the effect of superposition on erosion.

In [ ]:
from skimage import morphology, draw
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'

circle_left_img = np.zeros(shape=(300, 300), dtype=np.uint8)
circle_left_img[draw.disk((120, 120), 50)] = 1

circle_right_img = np.zeros_like(circle_left_img)
circle_right_img[draw.disk((180, 180), 50)] = 1


kernel = morphology.square(20)
plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)
plt.imshow(circle_left_img)
plt.axis('off')
plt.title(r'$g_1$')
plt.subplot(2, 3, 2)
plt.imshow(circle_right_img)
plt.axis('off')
plt.title(r'$g_2$')
plt.subplot(2, 3, 3)
plt.imshow(circle_left_img | circle_right_img)
plt.axis('off')
plt.title(r'$g_1 \vee g_2$')
plt.subplot(2, 3, 4)
plt.imshow(morphology.binary_erosion(circle_left_img | circle_right_img, footprint=kernel))
plt.axis('off')
plt.title('Erosion after superposition')
plt.subplot(2, 3, 5)
plt.imshow(morphology.binary_erosion(circle_left_img, footprint=kernel) 
           | morphology.binary_erosion(circle_right_img, footprint=kernel))
plt.axis('off')
plt.title('Erosion before superposition')
plt.show()

### c) Chaining

Show that *dilation* and *erosion* have the following properties: given two structering elements $S_1$ and $S_2$, it holds

\begin{align}
  (g\oplus S_1)\oplus S_2 & & = & g\oplus (S_1\oplus S_2) && = (g\oplus S_2)\oplus S_1 \\
  (g\ominus S_1)\ominus S_2 & & = & g\ominus (S_1\ominus S_2) && = (g\ominus S_2)\ominus S_1 \\  
\end{align}

What are the practical consequences?

Proof: First, by straight forward calculation we get (again using the simplified notation from part a):
\begin{align*}
  ((g\oplus S_1)\oplus S_2)(x,y)
  &= \bigvee_{(k,l)\in S_2}(g\oplus S_1)(x+k,y+l) \\
  &= \bigvee_{(k,l)\in S_2}\bigvee_{(i,j)\in S_1}g(x+k+i,y+l+j) \\
  &= \bigvee_{(i,j)\in S_1}\bigvee_{(k,l)\in S_2}g(x+k+i,y+l+j) \\
  &= \bigvee_{(i,j)\in S_1}(g\oplus S_2)(x+i,y+j) \\
  &= ((g\oplus S_2)\oplus S_1)(x,y)
\end{align*}  
so we see that the left and right side of the first row agree. To see that also the middle term is equivalent, note that $(k+i,l+j)\in S_1\oplus S_2$ is equivalent to $(i,j)\in S_1$ and $(k,l)\in S_2$. The second row is analogous, replacing $\bigvee$ by $\bigwedge$.

Practically this means, if one is going to erode or dilate an image with different structuring elements, 
1. the order of application does not matter
2. it is usually more efficient to form a combined structural element by dilation and the apply this combined element to the image.

Remark: for the argument to work, we have to assume, that images and structuring elements are large enough and zero-padded. Otherwise it is possible to create counter examples like the following:

In [ ]:
import numpy as np
from scipy.ndimage import binary_dilation
a =          np.array([[0,0,0,0,0],
                      [0,0,0,0,0],
                      [0,0,1,0,0],
                      [0,0,0,0,0],
                      [0,0,0,0,0]])
b2 =          np.array([[0,0,0,0,0], #for this one the rule holds 
                      [0,1,1,1,0],
                      [0,1,1,1,0],
                      [0,1,1,1,0],
                      [0,0,0,0,0]])
b =          np.array([[1,1,1], #for this one it dosen´t
                      [1,1,1],
                      [1,1,1],])
c = b
c2 = b2
d = np.zeros(c.shape, int)
out1 = np.zeros(a.shape, int)
out2 = np.zeros(a.shape, int)
out1 = binary_dilation(binary_dilation(a,b),c)
out2 = binary_dilation(a,binary_dilation(b,c))

print(out1)
print(out2)

## Assignment 2: Application (5 points)


### a) Boundary extraction

Extract the boundary of a shape using opening or closing. You may use `binary_dilation` or `binary_erosion` from `scipy.ndimage`. Can you achieve a thicker boundary?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
from scipy.ndimage import binary_erosion
from imageio.v2 import imread

def my_boundary(img):
    """
    Compute boundary of binary image.

    Parameters
    ----------
    img : ndarray of bools
        A binary image.
        
    Returns
    -------
    boundary : ndarray of bools
        The boundary as a binary image.
    """
    
    boundary = np.zeros(img.shape, bool)
    # BEGIN SOLUTION
    boundary = img ^ binary_erosion(img, iterations=5)
    # END SOLUTION
    
    return boundary
    
img = imread("images/engelstrompete.png") > 0
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(my_boundary(img))
plt.show()

### b)  Distance transform

Implement distance transform according to the ideas of (CV-05 slides 34ff).  Discuss the effect of different structuring elements.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import generate_binary_structure, binary_erosion
from imageio.v2 import imread

def my_distance_transform(img):
    """Distance transform of binary image.

    Args:
    img (ndarray of bools): A binary image.
        
    Returns:
    dt (ndarray of ints): The distance transform of the input image.
    """
    
    dt = np.zeros(img.shape,np.int32)
    # BEGIN SOLUTION
    while img.any():
        dt += img
        # generate_binary_structure(2, 1) -> N4 neighborhood -> city block distance
        # generate_binary_structure(2, 2) -> N8 neighborhood -> chessboard distance
        img = binary_erosion(img, generate_binary_structure(2, 2))
    # END SOLUTION

    return dt


img = imread("images/engelstrompete.png") > 0
plt.figure(figsize=(10,10))
plt.imshow(my_distance_transform(img) + 50 * img)
plt.show()

One can use different structuring elements to realize a different metric:

* $\begin{pmatrix} 0 & 1 & 0 \\ 1 & 1 & 1 \\ 0 & 1 & 0 \end{pmatrix}$ (the $N_4$-neighborhood) realizes the cityblock distance ($L_1$)
* $\begin{pmatrix} 1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{pmatrix}$ (the $N_8$-neighborhood) realizes the chessboard distance ($L_\infty$)



### c) Morphing

Write a function `my_morph` that implements morphing according to (CV-05 slide 41). You may use your function `my_distance_transform` from part b), or the function `distance_transform_edt` from `scipy.ndimage`.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
from imageio.v2 import imread


def my_morph(A,B,ratio):
    """Morphing from binary image A to image B.

    Args:
        A (ndarray of bools): A binary image (start).
        B (ndarray of bools): A binary image (target), same shape as A.
        ratio (float from 0.0 to 1.0): The ratio of image A and image B.
            0.0=only image A, 1.0=only image B.
        
    Returns:
        morph (ndarray of bools): A binary intermediate image between A and B.
        
    """

    result = np.zeros(A.shape, bool)
    # BEGIN SOLUTION
    dA = distance_transform_edt(A) - distance_transform_edt(~A)
    dB = distance_transform_edt(B) - distance_transform_edt(~B)
    result = (1-ratio)*dA + ratio*dB > 0
    # END SOLUTION
    
    return result

img1 = imread("images/kreis.png") > 0
img2 = imread("images/engelstrompete.png") > 0


plt.figure(figsize=(10,10))
plt.gray()
for i, ratio in enumerate(np.linspace(0, 1, 6), 1):
    plt.subplot(2, 3, i)
    plt.imshow(my_morph(img1, img2, ratio))
    plt.axis('off')
plt.show()

In [ ]:
# If you want to see your morph as an animation, run this cell. 
# Close the output (press the blue "Stop interaction" button) once you are done!

# Due to some matplotlib problem you may have to restart your kernel!
%matplotlib notebook
import matplotlib.animation as animation
fig = plt.figure()

ims = []
for i, ratio in enumerate(np.linspace(0, 1, 24), 1):
    plt.axis('off')
    im = plt.imshow(my_morph(img1, img2, ratio), cmap='gray', animated=True)
    ims.append([im])  
    
ani = animation.ArtistAnimation(fig, ims + list(reversed(ims)), interval=100, blit=True)

fig.show()

## Assignment 3: Implementation: Skeletonization (5 points)

### a) Skeletonization with hit-or-miss

Explain in your own words, how the hit-or-miss operator can be used for skeletonization (cf CV-05 slide 49).  

The hit-or-miss operator is used to detect small patterns in an image. To form the skeleton of an object in a binary image, one has to avoid to disconnect the "bones". The idea is to iteratively "eat" pixels from the boundary of the figure, but only in a way that will not break connectivity. The trick is to define a set of hit-or-miss operators that will detect boundary pixels that can be eaten without harm: they are all designed in a way, that they detect  a connected structures that stay connected even after removing the central pixel. Repeated application of these hit-and-miss operators will finally leave only the skeleton.

### b) Implementation of skeletonization

Now use this method to implement your own skeletonization function. It is ok to use
`scipy.ndimage.morphology.binary_hit_or_miss` here (but of course *not* `skimage.morphology.skeletonize` or similar functions). Compare your result with (CV-05 slide 50). Note that computing the skeleton using this method may take some time ...

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import binary_hit_or_miss, distance_transform_cdt, binary_dilation
from imageio.v2 import imread

def my_skeletonize(img):
    """
    Compute the skeloton of a binary image using hit_or_miss operator.
    
    Parameters
    ----------
    img : ndarray of bools
        Binary image to be skeletonized.
    
    Returns
    -------
    skeleton : ndarray of bools
        The skeleton of the input image.
    """
    # BEGIN SOLUTION
    skeleton = np.zeros(img.shape, bool)
    
    sl_hit = np.asarray([[0,0,1],[0,1,1],[0,0,1]], bool)
    sl_miss = np.asarray([[1,0,0],[1,0,0],[1,0,0]], bool)

    sr_hit = np.asarray([[1,0,0],[1,1,0],[1,0,0]], bool)
    sr_miss = np.asarray([[0,0,1],[0,0,1],[0,0,1]], bool)

    so_hit = np.asarray([[0,0,0],[0,1,0],[1,1,1]], bool)
    so_miss = np.asarray([[1,1,1],[0,0,0],[0,0,0]], bool)

    su_hit = np.asarray([[1,1,1],[0,1,0],[0,0,0]], bool)
    su_miss = np.asarray([[0,0,0],[0,0,0],[1,1,1]], bool)

    slu_hit = np.asarray([[0,0,0],[0,1,1],[0,1,1]], bool)
    slu_miss = np.asarray([[1,1,0],[1,0,0],[0,0,0]], bool)

    slo_hit = np.asarray([[0,1,1],[0,1,1],[0,0,0]], bool)
    slo_miss = np.asarray([[0,0,0],[1,0,0],[1,1,0]], bool)

    sru_hit = np.asarray([[0,0,0],[1,1,0],[1,1,0]], bool)
    sru_miss = np.asarray([[0,1,1],[0,0,1],[0,0,0]], bool)

    sro_hit = np.asarray([[1,1,0],[1,1,0],[0,0,0]], bool)
    sro_miss = np.asarray([[0,0,0],[0,0,1],[0,1,1]], bool)

    while (skeleton != img).any():
        skeleton = img
        img = img ^ binary_hit_or_miss(img, sl_hit, sl_miss)
        img = img ^ binary_hit_or_miss(img, sr_hit, sr_miss)
        img = img ^ binary_hit_or_miss(img, so_hit, so_miss)
        img = img ^ binary_hit_or_miss(img, su_hit, su_miss)

        img = img ^ binary_hit_or_miss(img, slo_hit, slo_miss)
        img = img ^ binary_hit_or_miss(img, slu_hit, slu_miss)
        img = img ^ binary_hit_or_miss(img, sro_hit, sro_miss)
        img = img ^ binary_hit_or_miss(img, sru_hit, sru_miss)
    
    return skeleton
    # END SOLUTION


img = imread("images/engelstrompete.png") > 0
skel = my_skeletonize(img)
result = distance_transform_cdt(img, metric='taxicab') + (50 * img)
result[binary_dilation(skel)] = 0
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(result)
plt.show()

## Assignment 4: Custom Structuring Element (5 points)

Landsat 7 is a satelite mission for acquisition of satellite imagery of Earth. Unfortunately the Scan Line Corrector failed, resulting in black stripes on the aquired images. More information: [https://www.usgs.gov/landsat-missions]


### a) A first fix

A rather crude fix is to apply a custom structuring element for dilation and erosion (see CV-05, 24ff). Complement the code below (in part (b)) in the following way:
* Rotate the image such that the gaps are horizontal.
* Dilate the rotated image with a vertical structuring element. I.e. take the maximum of an area of size $7 \times1$ and assign it to the center pixel. Repeat for all pixels.
* Erode the dilated image.
* Rotate the result back.

Remark: this exercise applies morphological operator to color images. This extends the idea of generalizing morphological operators to gray value images (CV-05, slide 51).  

### b) Improving the solution
You may get better results by thresholding and applying the morphological operations only to pixels below a threshold, i.e. gap pixels. Compliment your solution from a). 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimg
from skimage import color
from skimage.transform import rescale as rescale
from imageio.v2 import imread


angle = 15
thresh = .3
struc_elem = np.ones((5,1), dtype=bool)

img = imread("images/landsat_stack2.png")
img2 = img.copy()
img3 = img.copy()

# BEGIN SOLUTION
img = ndimg.rotate(img, angle)
img2 = img.copy()
img3 = img.copy()
img_gray = color.rgb2gray(img)
thresh_img = np.zeros_like(img_gray).astype(bool)

struc_elem_height_half = len(struc_elem)//2

# threshold such that only for black pixels the value is computed
thresh_img[img_gray < thresh] = True
thresh_img = ndimg.binary_dilation(thresh_img, iterations=1, structure=np.ones((5,5),dtype=bool))

# Dilation
for i in np.arange(struc_elem_height_half, img.shape[0] - struc_elem_height_half):
    for j in np.arange(0, img.shape[1]):
        if (thresh_img[i,j]):
             img2[i,j,:] = np.max(img[i-struc_elem_height_half:i+struc_elem_height_half,j,:], axis=0)

# Erosion
for i in np.arange(struc_elem_height_half, img.shape[0] - struc_elem_height_half):
    for j in np.arange(0, img.shape[1]):
        if (thresh_img[i,j]):
            img3[i,j,:] = np.min(img2[i-struc_elem_height_half:i+struc_elem_height_half,j,:], axis=0) 
                  
                        
img = ndimg.rotate(img, -angle)
img2 = ndimg.rotate(img2, -angle)
img3 = ndimg.rotate(img3, -angle)
# END SOLUTION
img = (img - np.min(img))/np.ptp(img)
img3 = (img3 - np.min(img3))/np.ptp(img3)


plt.figure(figsize=(15,45))
plt.subplot(3,1,1); plt.imshow(img); plt.axis('off')
plt.subplot(3,1,2); plt.imshow(img3); plt.axis('off')
plt.subplot(3,1,3); plt.imshow(thresh_img); plt.axis('off')
plt.imshow(thresh_img)
plt.show()


### c) Bonus
Can you think of (and implement) other ways to add the missing data? 


## Bonus: Painting with a webcam using color detection

If you solve this assignment you may leave out one of the other assignments. There will be similar assignments on most of the following sheets. These bonus assignments are intended to show potential applications of the techniques you learnt in class. They are usually a bit more challenging and often there exist multiple ways to address them. Even if you do not intend to solve them, you may profit from at least taking a look.

### Testing your webcam: Images
From now on we will try to make the exercises a bit more interactive and use live feed from your webcam. Unfortunately, using the webcam may not always work out of box (depending on your hardware/os configuration). So first make sure that you can grab an image from the webcam.

1. Use the `imageio` library as presented in the tutorial sessions. You will probably need to install `ffmpeg` packages as shown in the tutorial code.
1. Use the `cv2` library (opencv will use `gstreamer`). You will probably need to install then `opencv` package.

Hint: Sometimes it helps to restart the kernel.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# number of your webcam (usually 0)
webcam = 1

# Set this flag to either use "imageio" or "cv2"
use_imageio = False

if use_imageio:
    # use imageio for accessing the webcam (requires ffmpeg to be installed on your computer)
    import imageio
    try:
        with imageio.get_reader(f'<video{webcam}>') as reader:
            img = reader.get_next_data()
            ok = True
    except:
        ok = False
else:
    # use opencv for accessing the webcam
    import cv2
    try:
        camera = cv2.VideoCapture(webcam)
        ok, img = camera.read()
        if ok:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    finally:
        camera.release()

if ok:
    plt.imshow(img)
    plt.show()
else:
    print("Accessing your webcam failed.")

### Testing your webcam: Video

You can now test your webcam with video. A video stream is essentially a loop that repeatedly fetches new images and displays them. We will provide two code skeletons for such a loop, one for `imageio` and one for `cv2`(OpenCV), that you may use for your code. Make sure, you have set your `webcam` variable to a valid camera index.

**imageio**

For `imageio` you may use the following code:

In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import imageio

try:
    display(HTML("press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt) to stop the demo!"))
    with imageio.get_reader(f'<video{webcam}>') as reader:
        fig = plt.figure(figsize=(8,6))
        mpl_image = plt.imshow(reader.get_next_data())

        while True:
            img = reader.get_next_data()
            mpl_image.set_data(img)
            fig.canvas.draw()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    reader.close()
    # plt.close(fig)
    print("Camera was closed.")

**OpenCV**

For `cv2` you may use the following code:

In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import cv2
       
try:
    display(HTML("press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt) to stop the demo!"))
    camera = cv2.VideoCapture(webcam)
    ok, img = camera.read()
    if ok:
        fig = plt.figure(figsize=(8,6))
        mpl_image = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    while ok:
        mpl_image.set_data(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        fig.canvas.draw()
        ok, img = camera.read()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    camera.release()
    # plt.close(fig)
    print("Camera was closed.")

### a) Color detection
In this task we will track a small colored object (like the cap of a pen) in front of a neutral background of a different color. We will use the location of the object to paint on a virtual canvas. For that you have to implement the following tasks in the `draw_func` function:

* Convert the image `img` given to the `draw_func` into HSV color space. 
* Measure the color of your object. You may return the converted image and interactively measure the color with your mouse. Define your measured hue value in a constant
* Discard all channel except the hue channel. 
* Find the location with the most similar hue to the measured hue of your object.
* Paint a marker, for example a circle, at this position in `img_draw`.

In [ ]:
%matplotlib inline

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from skimage.color import rgb2hsv
from skimage.draw import disk

# Adapt this hue value to the hue of your object
hue = .3

# A global canvas to draw on
canvas = np.zeros((480,640,3), np.uint8) 

# radius and color of the brush
radius = 5
color = (255,255,255)

# saturation threshold for object
thresh = .2

def draw_func(img):
    """
    Draw a circle on img_draw at the detected object location.
    
    Args:
        img          the RGB input image (uint8)

    Returns:
        img_draw     img with circle drawn at postion of object
    """
    global canvas, hue, radius, color
    
    # BEGIN SOLUTION
    
    img_hsv = rgb2hsv(img)

    hue_channel = img_hsv[...,0]
    img_diff1 = np.abs(hue_channel - hue)
    img_diff2 = 1 - np.abs(hue_channel - hue)
    img_diff = np.minimum(img_diff1, img_diff2)
    
    minloc = np.unravel_index(np.argmin(img_diff), img_diff.shape)
    
    # check if object is saturated
    if img_hsv[minloc[0], minloc[1],1] > thresh:
        rr, cc = disk((minloc[0], minloc[1]), radius, shape=canvas.shape)
        canvas[rr, cc] = color
    # END SOLUTION
    
    return canvas

# Make a figure and axes with dimensions as desired.
fig = plt.figure(figsize=(8, 1))
ax = fig.add_axes([0.05, 0.80, 0.9, 0.15])
cb = mpl.colorbar.ColorbarBase(ax, cmap=mpl.cm.hsv, orientation='horizontal',
                               norm=mpl.colors.Normalize(vmin=0, vmax=1))
cb.set_ticks([hue])
cb.set_label('the hue value')
plt.show()

First test your function with single image. You may either grab an image from your webcam (as described above), or choose an arbitrary image from wherever you like

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

draw_func(img)
plt.subplot(1,2,1); plt.imshow(img)
plt.subplot(1,2,2); plt.imshow(canvas)
plt.show()

### b) Implement the paint functionality

Now run your function on video, interatively updateing the image.

In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import imageio

try:
    display(HTML("press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt) to stop the demo!"))
    with imageio.get_reader(f'<video{webcam}>') as webcam:
        fig = plt.figure(figsize=(12,6))
        plt.subplot(1,2,1)
        mpl_image1 = plt.imshow(webcam.get_next_data())
        plt.subplot(1,2,2)
        mpl_image2 = plt.imshow(webcam.get_next_data())

        while True:
            img = webcam.get_next_data()
            # mirror the image to make drawing easier
            img = img[:,::-1,:]
            img_processed = draw_func(img)
            mpl_image1.set_data(img)
            mpl_image2.set_data(img_processed)
            fig.canvas.draw()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    webcam.close()
    # plt.close(fig)
    print("Camera was closed.")